In [21]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [12]:
dfBefore = pd.read_csv('JamsBefore.csv')
dfBefore.head()

,date,flag,day,hour,minute,street,level,type,road_type,delay,speed_kmh,length,distance,count_data
0,2019-06-05,0,2.0,6,55,Gatot Subroto,3,0,7,119.142857,10.882857,510.0,1.131668,7
1,2019-06-05,0,2.0,7,26,Pelajar Pejuang 45,3,0,7,120.636364,9.958182,469.0,1.233164,11
2,2019-06-05,0,2.0,7,28,Jalan Jawa,4,0,1,384.750000,2.647500,296.5,1.716027,4
3,2019-06-05,0,2.0,7,39,Pelajar Pejuang 45,3,0,7,92.333333,8.795000,327.0,0.731429,6
4,2019-06-05,0,2.0,7,40,Jalan Sunda,4,0,7,109.500000,5.162500,194.0,1.313316,4


In [13]:
dfAfter = pd.read_csv('JamsAfter.csv')
dfAfter.head()

,date,flag,day,hour,minute,street,level,type,road_type,delay,speed_kmh,length,distance,count_data
0,2021-01-30,1,5.0,6,48,Laswi,3,0,7,69.846154,9.358462,330.0,0.897244,13
1,2021-01-30,1,5.0,6,52,WR Supratman,2,0,7,83.333333,18.913333,815.0,1.530744,3
2,2021-01-30,1,5.0,7,4,Jalan Sunda,2,0,7,61.625000,14.763750,584.0,1.281628,8
3,2021-01-30,1,5.0,7,10,Pelajar Pejuang 45,3,0,7,66.500000,23.055000,879.0,0.656896,2
4,2021-01-30,1,5.0,7,29,Pelajar Pejuang 45,3,0,7,77.000000,19.510000,781.0,0.979183,2


In [14]:
dfAfter = dfAfter.loc[(dfAfter['level'] != 5) & (dfAfter['level'] != 0)]
dfBefore = dfBefore.loc[(dfBefore['level'] != 5) & (dfBefore['level'] != 0)]
len(dfAfter), len(dfBefore)

(61364, 79539)

In [15]:
streetB, streetA = dfBefore['street'].unique(), dfAfter['street'].unique()
uniqueB = set(streetB) - set(streetA)
uniqueA = set(streetA) - set(streetB)
sameStreet = set(streetB) & set(streetA)
len(uniqueB), len(uniqueA), len(sameStreet)

(71, 17, 127)

Jalan yang terdapat sebelum dan sesudah pembangunan

In [22]:
featuresGreater = ['speed_kmh']
listDifference = []
listMeanBefore = []
listMeanAfter = []
for feature in featuresGreater:
    dictDifference = {}
    dictBefore = {}
    dictAfter = {}
    for street in sameStreet:
        data1 = dfBefore[feature].loc[dfBefore['street'] == street]
        data2 = dfAfter[feature].loc[dfAfter['street'] == street]

        dictBefore[street] = data1.mean()
        dictAfter[street] = data2.mean()
        
        # Hitung variance menggunakan Levene
        statsiticsVar, pValVar = stats.levene(data1, data2)
        # Tolong dikoreksi pemberian equalFlag jika ada kesalahan
        equalFlag = True
        if pValVar >= 0.05:
            equalFlag = False
    
        # Ketika menggunakan greater maka hipotesis data 1 adalah lebih besar dibandingkan data 2
        statsiticsTtest, pValTtest = stats.ttest_ind(data1,data2, equal_var = equalFlag, alternative = 'greater')
        # Tolong dikoreksi pemberian nilai T/F pada konsisi ini
        if pValTtest <= 0.05:
            dictDifference[street] = False
        else:
            dictDifference[street] = True
        
    listDifference.append(dictDifference)
    listMeanBefore.append(dictBefore)
    listMeanAfter.append(dictAfter)

TypeError: ttest_ind() got an unexpected keyword argument 'alternative'

In [18]:
dfStatistics = pd.DataFrame(sameStreet, columns= ['street']).set_index('street')
dfStatistics.head(2)

""
street
Jalan Sumbawa
Mutumanikam


In [19]:
for i in range(len(featuresGreater)):
    sentences = str(featuresGreater[i])
    sentStatistics = sentences + '_hypothesis'
    sentBefore = sentences + '_before'
    sentAfter = sentences + '_after'
    tempDF1 = pd.DataFrame.from_dict(listDifference[i], orient = 'index',  columns = [sentStatistics])
    tempDF2 = pd.DataFrame.from_dict(listMeanBefore[i], orient = 'index',  columns = [sentBefore])
    tempDF3 = pd.DataFrame.from_dict(listMeanAfter[i], orient = 'index',  columns = [sentAfter])
    
    dfStatistics = dfStatistics.merge(tempDF1, left_index = True, right_index = True)
    dfStatistics = dfStatistics.merge(tempDF2, left_index = True, right_index = True)
    dfStatistics = dfStatistics.merge(tempDF3, left_index = True, right_index = True)


In [20]:
dfStatistics

,speed_kmh_hypothesis,speed_kmh_before,speed_kmh_after
